In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Configuracion de carpetas
carpeta_drive_base = "drive/My Drive/Colab Notebooks/"
carpeta_drive_prueba_red = "drive/My Drive/Colab Notebooks/Prueba red/"
carpeta_drive_cifar = "drive/My Drive/Colab Notebooks/Prueba red/cifar_100_python/"

In [0]:
# Importaciones
from __future__ import print_function

import os
import sys

import tensorflow 
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras.datasets import cifar10, mnist


from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model
import cv2
import numpy as np
from matplotlib import pyplot as plt
import importlib.util
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator  

Using TensorFlow backend.


In [0]:
#Cargando modelo de filtros-feature
from keras.models import model_from_json

# json_file = open(carpeta_drive_prueba_red + 'model_selfsupervised_filter.json', 'r')
json_file = open(carpeta_drive_prueba_red + 'model_selfsupervised_filter_mnist.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
model_filter_feature = model_from_json(loaded_model_json)
# load weights into new model
model_filter_feature.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model_filter_feature.load_weights(carpeta_drive_prueba_red +"model_selfsupervised_filter.h5")
model_filter_feature.load_weights(carpeta_drive_prueba_red +"model_selfsupervised_filter_mnist.h5")

print("modelo cargado")
















modelo cargado


In [0]:
#Cargando modelo de rotacion-feature
# json_file = open(carpeta_drive_prueba_red + 'model.json', 'r')
json_file = open(carpeta_drive_prueba_red + 'model_mnist.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
model_rotation_feature = model_from_json(loaded_model_json)
# load weights into new model
model_rotation_feature.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model_rotation_feature.load_weights(carpeta_drive_prueba_red +"model.h5")
model_rotation_feature.load_weights(carpeta_drive_prueba_red +"model_mnist.h5")

print("modelo cargado")

modelo cargado


In [0]:
#Cargar modelo de rotacion-conjunto

# json_file = open(carpeta_drive_prueba_red + 'model_conjunto_rotacion_cifar.json', 'r')
json_file = open(carpeta_drive_prueba_red + 'model_conjunto_rotacion_mnist.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
model_rotation_conjunto = model_from_json(loaded_model_json)
# load weights into new model
model_rotation_conjunto.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model_rotation_conjunto.load_weights(carpeta_drive_prueba_red +"model_conjunto_rotacion_cifar.h5")
model_rotation_conjunto.load_weights(carpeta_drive_prueba_red +"model_conjunto_rotacion_mnist.h5")

print("modelo cargado")

modelo cargado


In [0]:
#Cargar modelo de filtros-conjunto
# json_file = open(carpeta_drive_prueba_red + 'model_selfsupervised_filter_conjunto.json', 'r')
json_file = open(carpeta_drive_prueba_red + 'model_selfsupervised_filter_conjunto_mnist.json', 'r')

loaded_model_json = json_file.read()
json_file.close()
model_filter_conjunto = model_from_json(loaded_model_json)
# load weights into new model
model_filter_conjunto.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model_filter_conjunto.load_weights(carpeta_drive_prueba_red +"model_selfsupervised_filter_conjunto.h5")
model_filter_conjunto.load_weights(carpeta_drive_prueba_red +"model_selfsupervised_filter_conjunto_mnist.h5")

print("modelo cargado")

modelo cargado


In [0]:
from keras import backend as K
def layers_extr(model,dat,name_model = "filter"):
  if(name_model == "filter"):
    inp = model.input                                           
    outputs = [layer.output for layer in model.layers[-2:]]         
    functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]    

    layer_outs = [[func([dat[i].reshape(1,32,32,3), 1.]) for func in functors] for i in range(len(dat))]
    features = [layer_outs[i][0][0][0] for i in range(len(layer_outs))]
  elif(name_model == "rotation"):
    inp = model.input                                           
    outputs = [layer.output for layer in model.layers[6:]]         
    functors = [K.function([inp, K.learning_phase()], [out]) for out in outputs]    

    layer_outs = [[func([dat[i].reshape(1,32,32,3), 1.]) for func in functors] for i in range(len(dat))]
    features = [layer_outs[i][0][0][0] for i in range(len(layer_outs))]
  return features


In [0]:
# (X_train, y_train), (X_test, y_test) = cifar10.load_data()
(X_train, y_train), (X_test, y_test) = mnist.load_data()


In [0]:
def extract_data_cifar(n_per_class,data_x,data_y):
    data_x_aux = []
    data_y_aux = []

    for i in range(10):
        aux = np.where(data_y == i)
        for cont in range(n_per_class):
            
            data_x_aux.append(data_x[aux[0][cont]])
            data_y_aux.append(data_y[aux[0][cont]])

            
    return(data_x_aux, data_y_aux)
import random
def des_ordenar_par(a,b):
    aux1 = []
    aux2 = []

    while len(a) > 0:
        num = random.randint(0,len(a)-1)
        aux1.append(a[num])
        aux2.append(b[num])

        del(a[num])
        del(b[num])
    return(aux1,aux2)

In [0]:
view_1 = model_rotation_conjunto
feature_1 = model_rotation_feature
view_2 = model_filter_conjunto
feature_2 = model_filter_feature

In [0]:
def coincidencia(view_1,view_2):
  datos_correlacion = []
  cont = 0
  while cont < len(view_1):
    if(np.argmax(view_1[cont][0][0]) == np.argmax(view_2[cont]) and max(view_1[cont][0][0]) > 0.80 and max(view_2[cont]) > 0.80):
      aux = [0 if i != np.argmax(view_2[cont]) else 1 for i in range(len(view_2[cont]))]
      datos_correlacion.append([cont,np.array(np.argmax(aux))])
    cont+=1
  return datos_correlacion

In [0]:
def parse_datos_ytrain(datos_correlacion,y_train):
  train_aux = y_train.tolist()
  for i in datos_correlacion:
    train_aux.append([i[1].tolist()])
  return train_aux
def parse_datos_xtrain(datos_correlacion,X_train,datos_n):
  train_aux = X_train.tolist()
  datos_n_aux = datos_n.tolist()
  for i in datos_correlacion:
    train_aux.append(datos_n_aux[i[0]])
  cont_del = 0
  for i in datos_correlacion:
    del(datos_n_aux[i[0] - cont_del])
    cont_del+=1
  return train_aux,datos_n_aux

In [0]:
def max_y_train(aux_y_train):
  aux_np_y_train = []
  for i in aux_y_train:
    if(len(i)>2):
      aux_np_y_train.append([(np.argmax(i))])
    else:
      aux_np_y_train.append(i)

  aux_np_y_train = np.array(aux_np_y_train)
  return aux_np_y_train

In [0]:
# Configuracion 
epoch = 10
batch = 250
verb = 0
iterator = 5

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
datagen.fit(X_train)
# dat = datagen.apply_transform(X_train[0],{'flip_horizontal':True,'flip_vertical':True,'theta':0.2})
dat = datagen.flow(X_train,batch_size=1)

(1, 32, 32, 3)

In [0]:
#Datos de entrada
X_train = X_train
datos_sin_etiqueta = 40000
datos_con_etiqueta = 500

datos_n = X_train[-datos_sin_etiqueta:-1]
datos_e, train_y = extract_data_cifar(datos_con_etiqueta,X_train,y_train)

datos_e,train_y = des_ordenar_par(datos_e,train_y)
datos_e,train_y = np.array(datos_e),np.array(train_y)

# Primera ejecucion
print("Extrayendo Feautres")
filter1 = layers_extr(feature_1,datos_e,"rotation")
filter2 = layers_extr(feature_2,datos_e,"filter")
print("Feautres Extraidos")

filter1 = np.array(filter1)
filter2 = np.array(filter2)

filter1= filter1.reshape(filter1.shape[0],1,1,filter1.shape[1])
filter2= filter2.reshape(filter2.shape[0],1,filter2.shape[1])

train_y = to_categorical(train_y,num_classes=None, dtype='float32')

print("Comenzando entrenamiento View1")
train_y_view1 = train_y[:]
train_y_view1 = train_y_view1.reshape(train_y_view1.shape[0],1,1,train_y_view1.shape[1])
print(train_y_view1[:2])
view_1.fit([datos_e,filter1],train_y_view1,epochs=epoch,batch_size=batch, verbose=verb)
print("Termino entrenamiento View1")
print("Comenzando entrenamiento View2")
view_2.fit([datos_e,filter2],train_y,epochs=epoch,batch_size=batch, verbose=verb)
print("Termino entrenamiento View2")

# N Ejecuciones
cont = 1
while cont<=iterator and len(datos_n) > 50:
  print("Comenzando iteracion ",cont)
  print("Extrayendo Feature")
  filter1 = layers_extr(feature_1,datos_n,"rotation")
  filter2 = layers_extr(feature_2,datos_n,"filter")
  print("Feature Extraidos")
  filter1 = np.array(filter1)
  filter2 = np.array(filter2)

  filter1= filter1.reshape(filter1.shape[0],1,1,filter1.shape[1])
  filter2= filter2.reshape(filter2.shape[0],1,filter2.shape[1])
  
  result_view1 = view_1.predict([datos_n,filter1])
  result_view2 = view_2.predict([datos_n,filter2])
  
  print("Buscando Coincidencias")
  datos_u = coincidencia(result_view1,result_view2)
  print("Coincidencias Encontradas")


  train_y = parse_datos_ytrain(datos_u,train_y)
  train_y = np.array(train_y)

  datos_e,datos_n = parse_datos_xtrain(datos_u,datos_e,datos_n)
  datos_e = np.array(datos_e)
  datos_n = np.array(datos_n)

  print("Extrayendo Feature")
  filter1 = layers_extr(feature_1,datos_e,"rotation")
  filter2 = layers_extr(feature_2,datos_e,"filter")
  print("Feature Extraidos")

  filter1 = np.array(filter1)
  filter2 = np.array(filter2)

  filter1= filter1.reshape(filter1.shape[0],1,1,filter1.shape[1])
  filter2= filter2.reshape(filter2.shape[0],1,filter2.shape[1])

  train_y = max_y_train(train_y)
  train_y = to_categorical(train_y,num_classes=None, dtype='float32')

  print("Comenzando entrenamiento View1")
  train_y_view1 = train_y[:]
  train_y_view1 = train_y_view1.reshape(train_y_view1.shape[0],1,1,train_y_view1.shape[1])
  print(train_y_view1[:2])
  view_1.fit([datos_e,filter1],train_y_view1,epochs=epoch,batch_size=batch, verbose=verb)
  print("Termino entrenamiento View1")
  print("Comenzando entrenamiento View2")
  view_2.fit([datos_e,filter2],train_y,epochs=epoch,batch_size=batch, verbose=verb)
  print("Termino entrenamiento View2")
  cont+=1


Extrayendo Feautres
Feautres Extraidos
Comenzando entrenamiento View1
[[[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]]


 [[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]]]
Termino entrenamiento View1
Comenzando entrenamiento View2
Termino entrenamiento View2
Comenzando iteracion  1
Extrayendo Feature
Feature Extraidos
Buscando Coincidencias
Coincidencias Encontradas
Extrayendo Feature
Feature Extraidos
Comenzando entrenamiento View1
[[[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]]


 [[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]]]
Termino entrenamiento View1
Comenzando entrenamiento View2
Termino entrenamiento View2
Comenzando iteracion  2
Extrayendo Feature
Feature Extraidos
Buscando Coincidencias
Coincidencias Encontradas
Extrayendo Feature
Feature Extraidos
Comenzando entrenamiento View1
[[[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]]


 [[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]]]
Termino entrenamiento View1
Comenzando entrenamiento View2
Termino entrenamiento View2
Comenzando iteracion  3
Extrayendo Feature
Feature Extraidos
Buscando Coincidencia

In [0]:
len(datos_n)

15806

In [0]:
muestra_i =5000
muestra_f = 10000
, 
# test_y = y_train[muestra_i:muestra_f]
# test_x = X_train[muestra_i:muestra_f]

test_y = y_test[:]
test_x = X_test[:]

print("Extrayendo Feature")
filter1_test = layers_extr(feature_1,test_x,"rotation")
filter2_test = layers_extr(feature_2,test_x,"filter")
print("Feature Extraidos")
filter1_test = np.array(filter1_test)
filter2_test = np.array(filter2_test)

filter1_test= filter1_test.reshape(filter1_test.shape[0],1,1,filter1_test.shape[1])
filter2_test= filter2_test.reshape(filter2_test.shape[0],1,filter2_test.shape[1])



Extrayendo Feature
Feature Extraidos


In [0]:
# filter1_test= filter1_test.reshape(filter1_test.shape[0],1,1,128)
result_view1 = view_1.predict([test_x,filter1_test])
result_view2 = view_2.predict([test_x,filter2_test])

In [0]:
cont = 0
for i in range(len(result_view1)):
  if(test_y[i][0] == np.argmax(result_view1[i][0][0])):
    cont+=1
print(cont/len(result_view1))
cont = 0
for i in range(len(result_view2)):
  if(test_y[i][0] == np.argmax(result_view2[i])):
    cont+=1
print(cont/len(result_view2))

'''
Registro mnist
1:
view 1: 0.7417
view 2: 0.6408

2:5 iteraciones
view 1: 0.9203
view 2: 0.9139

3:10 iteraciones
view 1: 0.8724
view 2: 0.8142
'''
'''
Registro cifar-10
1:
view 1: 0.8702
view 2: 0.8654
'''

0.7417
0.6408


In [0]:
# serialize model to JSON
model_json = view_1.to_json()
with open(carpeta_drive_prueba_red + "modelo_prueba_view1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
view_1.save_weights(carpeta_drive_prueba_red + "modelo_prueba_view1.h5")
print("Saved model to disk")

model_json = view_2.to_json()
with open(carpeta_drive_prueba_red + "modelo_prueba_view2.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
view_2.save_weights(carpeta_drive_prueba_red + "modelo_prueba_view2.h5")
print("Saved model to disk")

In [0]:
test_y = to_categorical(test_y,num_classes=None, dtype='float32')
test_y = test_y.reshape(test_y.shape[0],1,1,10)
view_1.fit([test_x,filter1_test],test_y,epochs=10,batch_size=batch, verbose=1)

Epoch 1/10
10000/10000 [==============================] - 9s 885us/step - loss: 1.3758 - acc: 0.6196
Epoch 2/10
10000/10000 [==============================] - 5s 500us/step - loss: 0.8039 - acc: 0.7615
Epoch 3/10
10000/10000 [==============================] - 5s 505us/step - loss: 0.6061 - acc: 0.8217
Epoch 4/10
10000/10000 [==============================] - 5s 510us/step - loss: 0.4844 - acc: 0.8518
Epoch 5/10
10000/10000 [==============================] - 5s 511us/step - loss: 0.4055 - acc: 0.8758
Epoch 6/10
10000/10000 [==============================] - 5s 514us/step - loss: 0.3347 - acc: 0.8958
Epoch 7/10
10000/10000 [==============================] - 5s 508us/step - loss: 0.3032 - acc: 0.9067
Epoch 8/10
10000/10000 [==============================] - 5s 502us/step - loss: 0.2579 - acc: 0.9197
Epoch 9/10
10000/10000 [==============================] - 5s 499us/step - loss: 0.2103 - acc: 0.9349
Epoch 10/10
10000/10000 [==============================] - 5s 497us/step - loss: 0.2855 - a

In [0]:
from keras.applications import nasnet
from keras.applications.vgg19 import VGG19
from keras.applications.resnet_v2 import ResNet50V2

In [0]:
base_model.summary()

In [0]:
# nas1 = nasnet.NASNetLarge(input_shape=(32,32,3),include_top=False, weights=None, classes=10)
# nas2 = nasnet.NASNetLarge(input_shape=(32,32,3),include_top=False, weights=None, classes=10)

vgg1 = ResNet50V2(weights=None,input_shape=(32,32,3),include_top=True,classes=10)
vgg2 = ResNet50V2(weights=None,input_shape=(32,32,3),include_top=True,classes=10)

# model = Model(inputs=base_model.input, outputs=base_model.get_layer('block5_conv4 ').output)
# base_model.summary()
# x = base_model.get_layer('block5_pool').output
# x = Dense(512)(x)
# dense = Dense(units=10, kernel_initializer="he_normal",
#               activation="softmax")(x)

# vgg1 = Model(inputs=base_model.input, outputs=dense)
# vgg2 = Model(inputs=base_model.input, outputs=dense)


In [0]:
vgg1.summary()

In [0]:
import keras.backend as K
import math

vgg1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
train_x = X_train[:]
train_y = y_train[:]
train_y = np.array(train_y)
train_y = to_categorical(train_y,num_classes=None, dtype='float32')

train_y = train_y.reshape(train_y.shape[0],10)

hist_vgg1 = vgg1.fit(train_x,train_y,batch_size=1000,epochs=10)



Epoch 1/10
50000/50000 [==============================] - 41s 824us/step - loss: 1.8369 - acc: 0.3603
Epoch 2/10
50000/50000 [==============================] - 17s 344us/step - loss: 1.3183 - acc: 0.5219
Epoch 3/10
50000/50000 [==============================] - 17s 340us/step - loss: 1.1070 - acc: 0.6032
Epoch 4/10
50000/50000 [==============================] - 17s 338us/step - loss: 0.9617 - acc: 0.6597
Epoch 5/10
50000/50000 [==============================] - 17s 338us/step - loss: 0.8021 - acc: 0.7169
Epoch 6/10
50000/50000 [==============================] - 17s 340us/step - loss: 0.6744 - acc: 0.7623
Epoch 7/10
50000/50000 [==============================] - 17s 341us/step - loss: 0.5705 - acc: 0.8005
Epoch 8/10
50000/50000 [==============================] - 17s 340us/step - loss: 0.4723 - acc: 0.8335
Epoch 9/10
50000/50000 [==============================] - 17s 340us/step - loss: 0.3933 - acc: 0.8617
Epoch 10/10
50000/50000 [==============================] - 17s 339us/step - loss: 

In [0]:
result_vgg1 = vgg1.predict(train_x)
# result_vgg1 = result_vgg1[0][:,0,0]
# result_vgg1 = np.array(result_vgg1)

In [0]:
cont = 0
result_vgg2 = vgg2.predict(train_x)
for i in range(len(result_vgg2)):
  if(np.argmax(train_y[i]) == np.argmax(result_vgg2[i])):
    cont+=1
print(cont/len(result_vgg2))

0.1


In [0]:
pr = np.array([[1.0,0.0,0.0],[1.0,0.0,0.0],[1.0,0.0,0.0]])
pr2 = np.array([[0.9,0.2,0.3],[0.8,0.2,0.1],[0.9,0.5,0.1]])

from math import log
b = tf.convert_to_tensor(pr)
b2 = tf.convert_to_tensor(pr2)

a = -K.sum(b*K.log(b2))
a = K.log(tf.convert_to_tensor(2.0))
b = K.constant(b) if not K.is_tensor(b) else b
b2 = K.cast(b2, b.dtype)
action_prob = K.sum(b * b2, axis=1)
cross_entropy = K.log(action_prob)
loss = -K.sum(cross_entropy)
cee = K.sum(K.categorical_crossentropy(b,b2,from_logits=False))
# cee(pr,pr2)

cee.eval(session=tf.compat.v1.Session())
# discounted_rewards = K.placeholder(shape=[None, ])
# discounted_rewards

1.2711121071635647

In [0]:
from keras import losses
import tensorflow as tf

def presonalize_function(red_2_predict):

  
  def dif(y_true,y_pred):
    
    # ce1 = -K.sum(y_true*K.log(y_pred))
    # ce2 = -K.sum(y_pred*K.log(red_2_predict))

    # action_prob = K.sum(y_true * y_pred, axis=1)
    # cross_entropy = K.log(action_prob)
    # ce1 = -K.sum(cross_entropy)
    # _beta = 0.5
    # y_pred = K.clip(y_pred, K.epsilon(), 1)

    # (1) dynamically update the targets based on the current state of the model: bootstrapped target tensor
    # use predicted class proba directly to generate regression targets
    # y_true_update = _beta * y_true + (1 - _beta) * y_pred

    # (2) compute loss as always
    # ce1 = -K.sum(y_true * K.log(y_pred), axis=-1)
    # difer = K.sum(K.maximum(0.0,-1*abs(y_pred - red_2_predict)))
    # ce1 = losses.categorical_crossentropy(y_true, y_pred)
    # ce2 = losses.categorical_crossentropy(y_pred, red_2_predict)

    lam = 0
    
    # y_pred = K.constant(y_pred) if not K.is_tensor(y_pred) else y_pred
    # y_true = K.cast(y_true, y_pred.dtype)
    ce1 = K.sum(K.categorical_crossentropy(y_true, y_pred, from_logits=False))
    ce2 = K.sum(K.categorical_crossentropy(y_pred, red_2_predict, from_logits=False))

    
# SVHN, Cifar-100 y otra base de datos 
    return ce1 - ce2*lam
  #restar lambda * suma de la comparacion 
  #- de la cross entropy Y_pred * log(red2_predict)
  return dif

In [0]:
loss_data = [result_vgg1[i:i+200] for i in range(0,len(result_vgg1),200)]
# loss_data = tf.convert_to_tensor(loss_data)
# result_vgg1 = tf.convert_to_tensor(result_vgg1)
# dataset = tf.data.Dataset.from_tensor_slices(result_vgg1)
# dataset = dataset.batch(3)
# print(loss_data.shape)
# datagen = ImageDataGenerator()
# prueba = datagen.flow(train_x,batch_size=200)
# loss_data = loss_data.reshape(200,loss_data.shape[0],10)
loss_data = np.array(loss_data)
vgg2.compile(loss=presonalize_function(loss_data),optimizer='adam',metrics=['accuracy'])

In [0]:
loss_data.shape

(200, 100, 10)

In [0]:
#Test
'''
1: red1 imagnet
1: red2 random

2: red1 random
2: red2 imagnet

Probar 2 redes diferentes (Modelos)

Cambiar los pesos iniciales ya pre-entrenados

Probar self-supervised con otras bases de datos

'''
hist = vgg2.fit(train_x,train_y,batch_size=200,epochs=10)

Epoch 1/10
50000/50000 [==============================] - 53s 1ms/step - loss: 2.3842e-05 - acc: 0.1000
Epoch 2/10
50000/50000 [==============================] - 28s 559us/step - loss: 2.3842e-05 - acc: 0.1000
Epoch 3/10
50000/50000 [==============================] - 28s 552us/step - loss: 2.3842e-05 - acc: 0.1000
Epoch 4/10
50000/50000 [==============================] - 28s 554us/step - loss: 2.3842e-05 - acc: 0.1000
Epoch 5/10
50000/50000 [==============================] - 28s 554us/step - loss: 2.3842e-05 - acc: 0.1000
Epoch 6/10
50000/50000 [==============================] - 28s 552us/step - loss: 2.3842e-05 - acc: 0.1000
Epoch 7/10
50000/50000 [==============================] - 28s 551us/step - loss: 2.3842e-05 - acc: 0.1000
Epoch 8/10
50000/50000 [==============================] - 27s 550us/step - loss: 2.3842e-05 - acc: 0.1000
Epoch 9/10
50000/50000 [==============================] - 27s 544us/step - loss: 2.3842e-05 - acc: 0.1000
Epoch 10/10
50000/50000 [=======================